# Lib库解

In [1]:
import torch as tor

导个库先

## 自动微分

### 求导向量基础配置

其实我们在进行数据的操作其实是将不同类型的数据进行一定的配置，以此完成数据的不同类型操作。或者从一个更为特殊的说法来说其实我们就是对于数据进行一些定义。

> 例子：我们对于 $y = 2x^Tx$ 进行对于列向量$\vec{x}$的求导

In [2]:
x = tor.arange(4.0) #创建数据，注意这里的4.0是用于使得数据自动生成float型变量便于调用torch的函数(某些torch函数需要float型值才可以调用)
x

tensor([0., 1., 2., 3.])

In [3]:
x.requires_grad_(True) # 状态值调整函数，表明在计算中x是作为被微分数值(一般而言这个值是false)
x.grad                 # 进行求导操作函数，注意这里需要开启requires_grad_才可以进行使用，这个函数是专门求导开了一个空间来存求导的数据，在这里注意是因为我们的求导数据没有存入任何信息，所以求不出来！

这里是把彻底作为了一个求导变量，配置完成

In [98]:
y = 2 * tor.dot(x,x)
print(y)
y.backward()  # 相应导数表达式存入
print(x.grad) # 对应通道数据求导
x.grad.zero_()# 归零存储空间(保证可复现)

tensor(28., grad_fn=<MulBackward0>)
tensor([ 0.,  4.,  8., 12.])


这里其实就是y构建函数求导项，x去自动求导罢了

In [100]:
y = 2 * tor.dot(x,x)
y.backward()
# y.backward()         # 注意这里其实是因为我们的数据其实不会自动归零这个backward()标志位【当然做出来的初衷是放置被重复求导】
print(x.grad == 4 * x)
x.grad.zero_()

tensor([ True, False, False, False])


tensor([0., 0., 0., 0.])

求导结果正确！

In [101]:
x.grad.zero_()
y = x.sum()         # x1 + x2 + x3 + x4
y.backward()
x.grad

tensor([1., 1., 1., 1.])

### 高级配置

这里会教大家一些高级配置

In [102]:
x.grad.zero_()
y = x * x
u = y.detach()     # 其实这就是打包y，使得y中的参数被固定！
z = u * x          # 注意，这里u的那个x^2是被当作一个实参输入！不是作为个x的表达式输入

z.sum().backward()
print(x.grad)
print(x.grad == u)

tensor([0., 1., 4., 9.])
tensor([True, True, True, True])


In [104]:
# 与上一个做未退化对比
x.grad.zero_()
y = x * x
z = y * x          # x^3

z.sum().backward()
print(x.grad)
print(x.grad == u)

tensor([ 0.,  3., 12., 27.])
tensor([ True, False, False, False])


### 带有优势的流控制

即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度

In [109]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = tor.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

a.grad == d / a

tensor(True)

### 函数库

In [4]:
grad_value = tor.arange(4.0)
grad_value

tensor([0., 1., 2., 3.])

#### requires_grad_

In [5]:
grad_value.requires_grad_(True)

tensor([0., 1., 2., 3.], requires_grad=True)

- 函数名称：requires_grad_(BOOL)  【注意这种函数名最后是以"_"下划线为终止的其实都是一种参数赋值调整函数】
- 函数作用：用于设置变量的求导许可为准许
- 函数输入：BOOL布尔值(True 准许 : False 不准许)
- 函数输出：无(调整变量的属性为可以进行求导)
- 函数依赖；toch_tensor 【且变量要为float值型】

#### backward() && grad && grad.zero_()

In [65]:
function_func = 2 * tor.dot(grad_value,grad_value) # 创建函数2*x^2
function_func2 = tor.dot(grad_value,grad_value)    # 创建函数  x^2
print("这是原函数")
print(function_func)
# function_func2.backward()                          # 在探究backward和grad关系的时候做的一个小实验，这个可以看到backward存储关联的本质
function_func.backward()                           # 创建求导的式子结构(关于求导变量的)
grad_value.grad
print(grad_value.grad)                             # 进行求导
grad_value.grad.zero_()                            # 归零(.grad是个存储空间不会自动清零)


这是原函数
tensor(28., grad_fn=<MulBackward0>)
tensor([ 0.,  6., 12., 18.])


tensor([0., 0., 0., 0.])

- 函数名称：backward()
- 函数作用：创建一个这个函数对于变量x(被求导变量【或者可以说是下一级基础变量】的符号类求导式子)
- 函数输入：无(依赖型)
- 函数输出：无(放入一个寄存空间，这个寄存空间是与x.grad绑定，其实函数的backward其实就是在进行求出一个符号表达式加入到x.grad的存储空间中，形成一个新表达式)
- 函数依赖：torch_tensor(float型保证，并且这是一个函数型位置)

- 函数名称：grad
- 函数作用：求导，并且与内含原有值相加(这个其实是因为grad没有自动清零导致的)【注意这个内涵原有值相加你是需要进行循环(或者是不在这一轮段程序中及逆行才行的！你连续接着的两个grad无效果),但是我们可以发现我们不清零的其实是符号求导公式，而不是内部的值】
- 函数输入：无(依赖)
- 函数输出：函数依据求导符号表达式求出的导数
- 函数依赖：torch_tensor(float型，并且requires_grad_为True)

- 函数名称：grad.zero_()
- 函数作用：将存储空间内的符号表达式进行清空
- 函数输入：无(依赖)
- 函数输入：无(但清空存储空间)
- 函数依赖：torch_tensor(float型，并且requires_grad_为True)

##### 从backward grad一窥jupyter Notebook存储机制

![backward与grad的关系详解](img\backward_grad在torch中的实现的思考.png)

当然，为什么在jupyter中会产生我们进行多次数据读入的时候我们的数据就发生了变化，那其实是因为在jupyter中我们在编写的代码其实内存空间其实是分开的，也就是说我们在进行多次调用这个程序块的时候我们可以发现我们的数据存储位置其实是会发生改变的，也就是我们的程序的被判定是一个新的位置，或者说是程序寄存器其实是将我们的程序在保有原有的变量地址和值的情况下我们的程序数据其实是增加的！也就是我们存储程序的地址其实是发生了改变的！这就是我们的数据为什么多次jupyter执行时会发生叠加问题。

![jupyter_Notebook存储机制分析](img\jupyter_Notebook存储原理实现猜想.png)

In [72]:
c = 0

In [82]:
c = c + 2 # 多次点击取得递增结果就是上面的验证
c

20

#### sum作用

sum()现在出现在这些数据当中的作用其实十分简单，就是我们将分通道的不同$x_1, x_2, x_3……$和并成一个加的表达式(其实从电路上来说就是相关与不相关，所以原理还是信号与系统那一套)

更有趣的关联其实就是我们需要的是一个表达式子的综合传出！这个其实挺像电路上的逻辑写法的^_^

#### detach()

In [105]:
grad_value.grad.zero_()
function_func3 = grad_value * grad_value
function_u = function_func3.detach()
function_z = function_u * grad_value

function_z.sum().backward()
print(grad_value.grad)
print(grad_value.grad == function_u)

tensor([0., 1., 4., 9.])
tensor([True, True, True, True])


- 函数名称：detach()
- 函数作用：退化当前变量，将其的表达式变量转化为值变量，也即是说这个函数返回的只有一个当前变量带函数式子的值的结果，而不带变量
- 函数输入：无(依赖型)
- 函数输出：退化为值的一个系数torch_tensor
- 函数依赖：torch_tensor(float型保证，并且这是一个函数型位置)

> 例子

In [106]:
x.grad.zero_()
y = x * x
u = y.detach()     # 其实这就是打包y，使得y中的参数被固定！
z = u * x          # 注意，这里u的那个x^2是被当作一个实参输入！不是作为个x的表达式输入

z.sum().backward()
print(x.grad)
print(x.grad == u)

tensor([0., 1., 4., 9.])
tensor([True, True, True, True])


In [107]:
# 与上一个做未退化对比
x.grad.zero_()
y = x * x
z = y * x          # x^3

z.sum().backward()
print(x.grad)
print(x.grad == u)

tensor([ 0.,  3., 12., 27.])
tensor([ True, False, False, False])
